<a href="https://colab.research.google.com/github/elangbijak4/multi-agent-AI/blob/main/Simple_Respon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Simple Reflex Agent Demo for Google Colab / Jupyter
# Copy-paste this cell into Google Colab and run.
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, patches
from IPython.display import HTML, display

In [2]:
# --- Parameters ---
GRID_ROWS = 5
GRID_COLS = 7
DIRTY_PROB = 0.25  # probability a cell starts dirty
STEPS = 80         # number of steps in the simulation
np.random.seed(42)

# --- Environment setup ---
# 0 = clean, 1 = dirty, 2 = obstacle (optional)
env = np.zeros((GRID_ROWS, GRID_COLS), dtype=int)
random_mask = np.random.rand(GRID_ROWS, GRID_COLS) < DIRTY_PROB
env[random_mask] = 1

# Place a couple of obstacles for interest (optional)
env[1, 3] = 2
env[3, 5] = 2

# Agent initial state
agent_pos = [0, 0]  # row, col

# Simple reflex agent function
def simple_reflex_action(perception, position, step):
    """
    perception: value of the current cell (0 clean, 1 dirty, 2 obstacle)
    position: (r, c)
    step: current time step (int) - can be used to vary behavior
    returns: action string and new position (if moved)
    """
    r, c = position
    if perception == 1:
        return 'SUCK', (r, c)  # clean current cell
    # If current cell is clean, move according to a simple sweeping pattern
    if r % 2 == 0:  # even row: move right
        potential = (r, c+1)
    else:            # odd row: move left
        potential = (r, c-1)
    pr, pc = potential
    if not (0 <= pr < GRID_ROWS and 0 <= pc < GRID_COLS) or env[pr, pc] == 2:
        # try move down
        potential = (r+1, c)
        pr, pc = potential
        if not (0 <= pr < GRID_ROWS and 0 <= pc < GRID_COLS) or env[pr, pc] == 2:
            # try move up (fallback)
            potential = (r-1, c)
            pr, pc = potential
            if not (0 <= pr < GRID_ROWS and 0 <= pc < GRID_COLS) or env[pr, pc] == 2:
                # no move possible, stay
                return 'NO_OP', (r, c)
    return 'MOVE', potential

# Simulation records
positions = []
actions = []
env_history = []

# Run simulation
for t in range(STEPS):
    positions.append(tuple(agent_pos))
    env_history.append(env.copy())
    perception = env[agent_pos[0], agent_pos[1]]
    action, new_pos = simple_reflex_action(perception, tuple(agent_pos), t)
    actions.append(action)
    if action == 'SUCK':
        env[agent_pos[0], agent_pos[1]] = 0
    elif action == 'MOVE':
        # move agent to new_pos
        agent_pos = [new_pos[0], new_pos[1]]
    else:
        pass

# Append final state
positions.append(tuple(agent_pos))
env_history.append(env.copy())

# --- Visualization ---
fig, ax = plt.subplots(figsize=(8, 5))
ax.set_title("Simple Reflex Agent Demo — Grid World")
ax.set_xticks(np.arange(-0.5, GRID_COLS, 1))
ax.set_yticks(np.arange(-0.5, GRID_ROWS, 1))
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)

def draw_frame(i):
    ax.clear()
    ax.set_title(f"Step {i}: Action = {actions[i] if i < len(actions) else '-'}")
    ax.set_xticks(np.arange(-0.5, GRID_COLS, 1))
    ax.set_yticks(np.arange(-0.5, GRID_ROWS, 1))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.grid(True)
    grid = env_history[i]
    for r in range(GRID_ROWS):
        for c in range(GRID_COLS):
            cell = grid[r, c]
            if cell == 1:
                rect = patches.Rectangle((c-0.5, r-0.5), 1, 1, linewidth=0.5)
                ax.add_patch(rect)
                ax.text(c, r, "●", fontsize=14, ha='center', va='center')
            elif cell == 2:
                rect = patches.Rectangle((c-0.5, r-0.5), 1, 1, linewidth=0.5)
                ax.add_patch(rect)
                ax.add_patch(patches.Rectangle((c-0.3, r-0.3), 0.6, 0.6, linewidth=1))
            else:
                rect = patches.Rectangle((c-0.5, r-0.5), 1, 1, linewidth=0.2, fill=False)
                ax.add_patch(rect)
    pr, pc = positions[i]
    ax.plot(pc, pr, marker='o', markersize=20)
    ax.text(pc, pr-0.35, "Agent", ha='center')

ani = animation.FuncAnimation(fig, draw_frame, frames=len(actions), interval=400, repeat=False)
plt.close(fig)
display(HTML(ani.to_jshtml()))

# Print short action transcript
print("Action transcript (first 40 steps):")
for t, act in enumerate(actions[:40]):
    pos = positions[t]
    print(f"Step {t:2d}: Pos {pos} -> {act}")


Action transcript (first 40 steps):
Step  0: Pos (0, 0) -> MOVE
Step  1: Pos (0, 1) -> MOVE
Step  2: Pos (0, 2) -> MOVE
Step  3: Pos (0, 3) -> MOVE
Step  4: Pos (0, 4) -> SUCK
Step  5: Pos (0, 4) -> MOVE
Step  6: Pos (0, 5) -> SUCK
Step  7: Pos (0, 5) -> MOVE
Step  8: Pos (0, 6) -> SUCK
Step  9: Pos (0, 6) -> MOVE
Step 10: Pos (1, 6) -> SUCK
Step 11: Pos (1, 6) -> MOVE
Step 12: Pos (1, 5) -> MOVE
Step 13: Pos (1, 4) -> MOVE
Step 14: Pos (2, 4) -> MOVE
Step 15: Pos (2, 5) -> MOVE
Step 16: Pos (2, 6) -> MOVE
Step 17: Pos (3, 6) -> MOVE
Step 18: Pos (4, 6) -> MOVE
Step 19: Pos (3, 6) -> MOVE
Step 20: Pos (4, 6) -> MOVE
Step 21: Pos (3, 6) -> MOVE
Step 22: Pos (4, 6) -> MOVE
Step 23: Pos (3, 6) -> MOVE
Step 24: Pos (4, 6) -> MOVE
Step 25: Pos (3, 6) -> MOVE
Step 26: Pos (4, 6) -> MOVE
Step 27: Pos (3, 6) -> MOVE
Step 28: Pos (4, 6) -> MOVE
Step 29: Pos (3, 6) -> MOVE
Step 30: Pos (4, 6) -> MOVE
Step 31: Pos (3, 6) -> MOVE
Step 32: Pos (4, 6) -> MOVE
Step 33: Pos (3, 6) -> MOVE
Step 34: Pos